In [13]:
!pip install aiohttp

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import asyncio
import aiohttp
import requests
import time
import json
from itertools import islice

In [2]:
def chunked_iterable(iterable, chunk_size):
    it = iter(iterable)
    while True:
        chunk = list(islice(it, chunk_size))
        if not chunk:
            break
        yield chunk

In [3]:
class timeit_context(object):
    def __init__(self):
        self.initial = None

    def __enter__(self):
        self.initial = time.time()

    def __exit__(self, type_arg, value, traceback):
        print('Total time elapsed {} sec'.format(time.time() - self.initial))

In [4]:
async def get_collection_attributes_async(base_url, nft_id_list):
    attributes = []
    max_workers = 200
    tcp_connection = aiohttp.TCPConnector(limit=max_workers)
    async with aiohttp.ClientSession(connector=tcp_connection) as session:
        for nft_id_chunk in chunked_iterable(nft_id_list, max_workers):
            results = await get_nft_attributes_of_chuck(base_url, nft_id_chunk, session)
            attributes.extend(results)
    
    await tcp_connection.close()
    return attributes

In [5]:
def get_collection_attributes_sync(base_url, nft_id_list):
    timeout = 10
    header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
                            'AppleWebKit/537.11 (KHTML, like Gecko) '
                            'Chrome/23.0.1271.64 Safari/537.11',
              'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
              'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
              'Accept-Encoding': 'none',
              'Accept-Language': 'en-US,en;q=0.8',
              'Connection': 'keep-alive'}
    
    attributes = []
    for nft_id in nft_id_list:
        response = requests.get(url=f'{base_url}/{nft_id}', headers=header, timeout=timeout)
        attributes.append(json.loads(response.text))
    return response

In [6]:
async def get_nft_attributes_of_chuck(base_url, nft_id_list, session):
    tasks = []
    
    for nft_id in nft_id_list:
        url = f'{base_url}/{nft_id}'
        task = asyncio.ensure_future(get_nft_attributes(url, session=session))
        tasks.append(task)
    
    return await asyncio.gather(*tasks, return_exceptions=True)

In [7]:
async def get_nft_attributes(url, session):
    timeout = 10
    try:
        async with session.get(url, timeout=timeout) as response:
            result = await response.text()
            return json.loads(result)
    except Exception as e:
        print(
            f"Fetcha could not get the data for NFT. Error message: {e}. URL : {url}")
        return {}

In [8]:
token_base_url = 'https://api.coolcatsnft.com/cat'
nft_id_start = 0
nft_id_end = 2000

with timeit_context():
    attributes = await get_collection_attributes_async(token_base_url, range(nft_id_start, nft_id_end))


Total time elapsed 11.485644102096558 sec


In [ ]:
with timeit_context():
    attributes = get_collection_attributes_sync(token_base_url, range(nft_id_start, nft_id_end))
